In [1]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fnmatch
from yelpapi import YelpAPI
import json
from decimal import *

%matplotlib inline

In [2]:
pov_data = pd.read_excel('../Data/poverty_rates.xlsx', sheet_name = 'Percents', header = 2)
pov_data

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,0.026037,0.012483,0.045000,0.004918,0.034508,0.011601,0.034091,0.008723,...,0.036052,0.009398,0.024596,0.005037,0.036945,0.007138,0.012765,0.003430,0.028517,0.009377
1,.50 to .74,poverty,0.024342,0.012483,0.023456,0.003069,0.022094,0.010641,0.016845,0.005842,...,0.019741,0.007608,0.021070,0.004877,0.014934,0.004207,0.010619,0.003978,0.011689,0.004958
2,.75 to .99,poverty,0.022202,0.008382,0.037121,0.003978,0.026306,0.010788,0.023047,0.007602,...,0.034112,0.011686,0.020271,0.004767,0.024564,0.005882,0.008348,0.003414,0.015594,0.005010
3,1.00 to 1.24,low-income,0.030584,0.012305,0.031564,0.003722,0.026528,0.010641,0.041693,0.013684,...,0.021880,0.006415,0.024117,0.004423,0.023726,0.005643,0.021285,0.006578,0.020476,0.006423
4,1.25 to 1.49,low-income,0.033081,0.010700,0.039468,0.004527,0.027562,0.009163,0.030210,0.010083,...,0.039582,0.011885,0.034043,0.007765,0.029409,0.006859,0.010087,0.002945,0.023045,0.006860
5,1.50 to 1.74,low-income,0.038698,0.012037,0.038541,0.004502,0.034360,0.010345,0.048496,0.013404,...,0.032074,0.009597,0.026598,0.005283,0.027834,0.005523,0.012201,0.003759,0.025152,0.007039
6,1.75 to 1.84,low-income,0.019082,0.012127,0.016473,0.002711,0.020838,0.011158,0.018326,0.008083,...,0.013526,0.007260,0.014006,0.004374,0.010906,0.003370,0.008896,0.002631,0.009223,0.003468
7,1.85 to 1.99,low-income,0.029157,0.010878,0.020719,0.003376,0.031848,0.011158,0.025768,0.008283,...,0.021929,0.006663,0.024473,0.004755,0.020656,0.005762,0.012358,0.006061,0.013976,0.003879
8,2.00 to 2.99,NaN,0.169327,0.021400,0.146650,0.007731,0.202172,0.029262,0.197423,0.023648,...,0.179712,0.022576,0.178449,0.014190,0.147064,0.012102,0.086676,0.009163,0.156998,0.018035
9,3.00 to 3.99,NaN,0.192867,0.024253,0.134462,0.007917,0.196483,0.031848,0.165773,0.021407,...,0.167330,0.022327,0.161986,0.009325,0.172346,0.014156,0.099018,0.010431,0.156073,0.015004


In [3]:
pov_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rate                   13 non-null     object 
 1   Income level           8 non-null      object 
 2   Cheatham County, TN    13 non-null     float64
 3   Error                  13 non-null     float64
 4   Davidson County, TN    13 non-null     float64
 5   Error.1                13 non-null     float64
 6   Dickson County, TN     13 non-null     float64
 7   Error.2                13 non-null     float64
 8   Maury County, TN       13 non-null     float64
 9   Error.3                13 non-null     float64
 10  Montgomery County, TN  13 non-null     float64
 11  Error.4                13 non-null     float64
 12  Robertson County, TN   13 non-null     float64
 13  Error.5                13 non-null     float64
 14  Rutherford County, TN  13 non-null     float64
 15  Error.6 

In [4]:
pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']] = pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']].apply(lambda x: x*100)
pov_data

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.603656,1.248328,4.499965,0.491754,3.450824,1.160127,3.409091,0.872279,...,3.605172,0.939831,2.459580,0.503710,3.694547,0.713787,1.276489,0.343008,2.851711,0.937725
1,.50 to .74,poverty,2.434240,1.248328,2.345583,0.306947,2.209414,1.064066,1.684539,0.584187,...,1.974142,0.760816,2.106983,0.487739,1.493371,0.420696,1.061914,0.397826,1.168945,0.495838
2,.75 to .99,poverty,2.220241,0.838163,3.712135,0.397752,2.630607,1.078844,2.304738,0.760243,...,3.411238,1.168573,2.027127,0.476682,2.456385,0.588177,0.834808,0.341441,1.559449,0.500976
3,1.00 to 1.24,low-income,3.058404,1.230495,3.156434,0.372173,2.652775,1.064066,4.169334,1.368438,...,2.187966,0.641472,2.411666,0.442282,2.372645,0.564251,2.128526,0.657823,2.047580,0.642277
4,1.25 to 1.49,low-income,3.308070,1.069996,3.946822,0.452746,2.756226,0.916279,3.020967,1.008323,...,3.958230,1.188463,3.404344,0.776451,2.940883,0.685874,1.008661,0.294454,2.304491,0.685952
5,1.50 to 1.74,low-income,3.869817,1.203745,3.854098,0.450188,3.436045,1.034508,4.849552,1.340429,...,3.207360,0.959722,2.659836,0.528281,2.783372,0.552288,1.220104,0.375899,2.515158,0.703936
6,1.75 to 1.84,low-income,1.908159,1.212662,1.647280,0.271136,2.083795,1.115791,1.832586,0.808259,...,1.352561,0.726007,1.400560,0.437368,1.090619,0.336955,0.889627,0.263129,0.922310,0.346830
7,1.85 to 1.99,low-income,2.915738,1.087829,2.071889,0.337641,3.184808,1.115791,2.576825,0.828265,...,2.192939,0.666335,2.447295,0.475453,2.065597,0.576214,1.235767,0.606137,1.397595,0.387935
8,2.00 to 2.99,NaN,16.932679,2.139991,14.665013,0.773122,20.217247,2.926180,19.742318,2.364757,...,17.971159,2.257583,17.844857,1.418989,14.706410,1.210248,8.667596,0.916253,15.699825,1.803515
9,3.00 to 3.99,NaN,19.286670,2.425323,13.446179,0.791666,19.648267,3.184808,16.577305,2.140685,...,16.732969,2.232720,16.198585,0.932478,17.234573,1.415612,9.901796,1.043119,15.607337,1.500360


In [5]:
pov_data.round(2)

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.60,1.25,4.50,0.49,3.45,1.16,3.41,0.87,...,3.61,0.94,2.46,0.50,3.69,0.71,1.28,0.34,2.85,0.94
1,.50 to .74,poverty,2.43,1.25,2.35,0.31,2.21,1.06,1.68,0.58,...,1.97,0.76,2.11,0.49,1.49,0.42,1.06,0.40,1.17,0.50
2,.75 to .99,poverty,2.22,0.84,3.71,0.40,2.63,1.08,2.30,0.76,...,3.41,1.17,2.03,0.48,2.46,0.59,0.83,0.34,1.56,0.50
3,1.00 to 1.24,low-income,3.06,1.23,3.16,0.37,2.65,1.06,4.17,1.37,...,2.19,0.64,2.41,0.44,2.37,0.56,2.13,0.66,2.05,0.64
4,1.25 to 1.49,low-income,3.31,1.07,3.95,0.45,2.76,0.92,3.02,1.01,...,3.96,1.19,3.40,0.78,2.94,0.69,1.01,0.29,2.30,0.69
5,1.50 to 1.74,low-income,3.87,1.20,3.85,0.45,3.44,1.03,4.85,1.34,...,3.21,0.96,2.66,0.53,2.78,0.55,1.22,0.38,2.52,0.70
6,1.75 to 1.84,low-income,1.91,1.21,1.65,0.27,2.08,1.12,1.83,0.81,...,1.35,0.73,1.40,0.44,1.09,0.34,0.89,0.26,0.92,0.35
7,1.85 to 1.99,low-income,2.92,1.09,2.07,0.34,3.18,1.12,2.58,0.83,...,2.19,0.67,2.45,0.48,2.07,0.58,1.24,0.61,1.40,0.39
8,2.00 to 2.99,NaN,16.93,2.14,14.67,0.77,20.22,2.93,19.74,2.36,...,17.97,2.26,17.84,1.42,14.71,1.21,8.67,0.92,15.70,1.80
9,3.00 to 3.99,NaN,19.29,2.43,13.45,0.79,19.65,3.18,16.58,2.14,...,16.73,2.23,16.20,0.93,17.23,1.42,9.90,1.04,15.61,1.50


In [6]:
with open('keys.json') as fi:
    credentials = json.load(fi)

In [7]:
api_key = credentials['yelp_key']
client_id = credentials['yelp_id']

In [12]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'park', location = 'Wilson County, TN', search_limit = 200)
response

{'businesses': [{'id': 'R31Aoy1vHCUiSbHYi1rcIw',
   'alias': 'don-fox-park-lebanon',
   'name': 'Don Fox Park',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/yFx37Us-AVaifZQ1F3idiw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/don-fox-park-lebanon?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 11,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 3.5,
   'coordinates': {'latitude': 36.2182684, 'longitude': -86.3116517},
   'transactions': [],
   'location': {'address1': '416 Baird Park Cir',
    'address2': '',
    'address3': '',
    'city': 'Lebanon',
    'zip_code': '37087',
    'country': 'US',
    'state': 'TN',
    'display_address': ['416 Baird Park Cir', 'Lebanon, TN 37087']},
   'phone': '+16154490303',
   'display_phone': '(615) 449-0303',
   'distance': 7064.107658925387},
  {'id': '6du6r19vgQ2eRmXb3r4M_g',
   'alias': 

In [13]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)
                    
data.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,R31Aoy1vHCUiSbHYi1rcIw,don-fox-park-lebanon,Don Fox Park,https://s3-media4.fl.yelpcdn.com/bphoto/yFx37U...,False,https://www.yelp.com/biz/don-fox-park-lebanon?...,11,"[{'alias': 'parks', 'title': 'Parks'}]",3.5,"{'latitude': 36.2182684, 'longitude': -86.3116...",[],"{'address1': '416 Baird Park Cir', 'address2':...",+16154490303,(615) 449-0303,7064.107659
1,6du6r19vgQ2eRmXb3r4M_g,cedars-of-lebanon-state-park-lebanon-4,Cedars Of Lebanon State Park,https://s3-media4.fl.yelpcdn.com/bphoto/z2txvE...,False,https://www.yelp.com/biz/cedars-of-lebanon-sta...,41,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.091612, 'longitude': -86.330965}",[],"{'address1': '328 Cedar Forest Rd', 'address2'...",+16154432769,(615) 443-2769,7391.371636
2,EbxNiXeQMGM84Gl5KDLahA,rock-island-state-park-rock-island-4,Rock Island State Park,https://s3-media3.fl.yelpcdn.com/bphoto/t3LetC...,False,https://www.yelp.com/biz/rock-island-state-par...,57,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 35.8100143074989, 'longitude': -8...",[],"{'address1': '82 Beach Rd', 'address2': None, ...",+19318374770,(931) 837-4770,70966.759737
3,DiG2uoaL8ZLQcpGlkHraag,long-hunter-state-park-hermitage,Long Hunter State Park,https://s3-media1.fl.yelpcdn.com/bphoto/KLoy5s...,False,https://www.yelp.com/biz/long-hunter-state-par...,38,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.0943884, 'longitude': -86.5571...",[],"{'address1': '2910 Hobson Pike', 'address2': '...",+16158852422,(615) 885-2422,23605.899650
4,Q9wU-l_5R9AbhbAGOujatw,sellars-state-farm-archaeological-area-lebanon,Sellars State Farm Archaeological Area,https://s3-media1.fl.yelpcdn.com/bphoto/8_5pAY...,False,https://www.yelp.com/biz/sellars-state-farm-ar...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.0,"{'latitude': 36.1654368105194, 'longitude': -8...",[],"{'address1': '433 Poplar Hill Rd', 'address2':...",,,5778.970145
5,tigsP5nwiJV8KmvC6OQ2_w,barfield-crescent-park-murfreesboro,Barfield Crescent Park,https://s3-media2.fl.yelpcdn.com/bphoto/aTmm7W...,False,https://www.yelp.com/biz/barfield-crescent-par...,24,"[{'alias': 'discgolf', 'title': 'Disc Golf'}, ...",5.0,"{'latitude': 35.7862282, 'longitude': -86.416069}",[],"{'address1': '697 Barfield Crescent Rd', 'addr...",+16158905333,(615) 890-5333,42178.817689
6,njenDpIerIT38E4lgIhjkA,gregory-mill-dam-park-smyrna,Gregory Mill Dam Park,https://s3-media2.fl.yelpcdn.com/bphoto/4v2vjK...,False,https://www.yelp.com/biz/gregory-mill-dam-park...,4,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 35.9739189, 'longitude': -86.5039...",[],"{'address1': '390 Enon Springs Rd E', 'address...",+16154599773,(615) 459-9773,26936.111173
7,nRbgiD7uTBqfV_yS8uVfAw,seven-points-recreation-area-hermitage,Seven Points Recreation Area,https://s3-media2.fl.yelpcdn.com/bphoto/zlVu0v...,False,https://www.yelp.com/biz/seven-points-recreati...,9,"[{'alias': 'boating', 'title': 'Boating'}, {'a...",4.5,"{'latitude': 36.1338654, 'longitude': -86.5704...",[],"{'address1': '1810 Stewarts Ferry Pike', 'addr...",+16158895198,(615) 889-5198,23926.939422
8,emWQlqb8Lnr9HKNNXP7jsQ,bledsoe-creek-state-park-gallatin-2,Bledsoe Creek State Park,https://s3-media3.fl.yelpcdn.com/bphoto/_uNcGk...,False,https://www.yelp.com/biz/bledsoe-creek-state-p...,18,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.379714, 'longitude': -86.356685}",[],"{'address1': '400 Zieglers Fort Rd', 'address2...",+16154523706,(615) 452-3706,25408.900958
9,eQB8iaJ5utCWnm4P2qUcCw,percy-priest-lake-nashville,Percy Priest Lake,https://s3-media2.fl.yelpcdn.com/bphoto/abQBy2...,False,https://www.yelp.com/biz/percy-priest-lake-nas...,14,"[{'alias': 'lakes', 'title': 'Lakes'}, {'alias...",4.5,"{'latitude': 36.156946, 'longitude': -86.6175}",[],"{'address1': '3737 Bell Rd', 'address2': '', '...